# KnitHAT 

This notebook is step four in the process of building [KnitHAT](http://knithat.works), which recommends hat patterns from Ravelry based on user-uploaded images. KnitHAT was my Insight Data Science fellowship project, and was built in only three weeks.

This code extracts pattern category and attribute information from the text scraped from Ravelry and creates one-hot encoding for each.

#### notes to self re: process

- import each csv
- make new df of the first patterns column only and the id column
- split first column out with regex to extract pattern category and pattern attributes fields into new columns 
    - don't expand them
- create columns with the pattern category and pattern attribute names
- write loops: if category/attributes appears in patterns column, add 1 to column for that category/attribute, else 0

In [1]:
import pandas as pd 
import re

In [57]:
which_file = 12000  # enter number from filename below

In [58]:
d = pd.read_csv('/Users/ssivek/Insight_Projects/Ravelry_Project/Data/cleaned_csv_files/newcsv12000.csv')

In [59]:
d = d.dropna(how='all') # drop rows where all values are NaN
d.head(5)

,patterns,pattern_id,pattern_name,medium2_urls,author_name,medium_url,remainder,extension,ids_urls
0,"{""patterns"": {""17016"":{""comments_count"":0,""cre...",17016,13505 Bonnet,https://images4-e.ravelrycache.com/uploads/mam...,Dale Design,https://images4-e.ravelrycache.com/uploads/mam...,"""small2_url"":""https://images4-b.ravelrycache.c...",jpg,"(17016, 'https://images4-e.ravelrycache.com/up..."
1,"{""patterns"": {""541305"":{""comments_count"":0,""cr...",541305,Alfa lue og hals,https://images4-b.ravelrycache.com/uploads/Gur...,Guri Østereng Halvorsen,https://images4-b.ravelrycache.com/uploads/Gur...,NaN,JPG,"(541305, 'https://images4-b.ravelrycache.com/u..."
2,"{""patterns"": {""279164"":{""comments_count"":0,""cr...",279164,F145 Heaven Ribbed Scarf,https://images4-e.ravelrycache.com/uploads/Ply...,JoAnne Turcotte,https://images4-e.ravelrycache.com/uploads/Ply...,NaN,jpg,"(279164, 'https://images4-e.ravelrycache.com/u..."
3,"{""patterns"": {""934524"":{""comments_count"":1,""cr...",934524,Garter Stitch Bonnet,NaN,Yarnspirations Design Studio,NaN,NaN,jpg,"(934524, nan, 'jpg')"
4,"{""patterns"": {""934526"":{""comments_count"":0,""cr...",934526,Baby Turban,NaN,Yarnspirations Design Studio,NaN,NaN,jpg,"(934526, nan, 'jpg')"


In [60]:
# d.drop([17], inplace = True) # row 17 in 1000 file has NaN

In [ ]:
# All this mess was for 1000 file with .0 behind pattern ID
# d['pattern_id_2'] = d['patterns'].str.extract('({"patterns": {"\([0-9]+\)")', expand = False)            
# pattern_id_3 = d["pattern_id"].str.split(".", n = 1, expand = True) 
# d.pattern_id.apply(str) appears to work but is object
# d['pattern_id'] = d['pattern_id'].str[:-1] # use if pattern_id imports with .0; requires string

# pd.to_numeric(d['pattern_id'], errors='coerce'); # changes to float64 but then doesn't let me do any operations on it

In [43]:
# d['pattern_id'] = d['pattern_id'].astype(str)
# d['pattern_id'] = d['pattern_id'].str.replace('.0', '') # finally

In [61]:
d.head() # clean

,patterns,pattern_id,pattern_name,medium2_urls,author_name,medium_url,remainder,extension,ids_urls
0,"{""patterns"": {""17016"":{""comments_count"":0,""cre...",17016,13505 Bonnet,https://images4-e.ravelrycache.com/uploads/mam...,Dale Design,https://images4-e.ravelrycache.com/uploads/mam...,"""small2_url"":""https://images4-b.ravelrycache.c...",jpg,"(17016, 'https://images4-e.ravelrycache.com/up..."
1,"{""patterns"": {""541305"":{""comments_count"":0,""cr...",541305,Alfa lue og hals,https://images4-b.ravelrycache.com/uploads/Gur...,Guri Østereng Halvorsen,https://images4-b.ravelrycache.com/uploads/Gur...,NaN,JPG,"(541305, 'https://images4-b.ravelrycache.com/u..."
2,"{""patterns"": {""279164"":{""comments_count"":0,""cr...",279164,F145 Heaven Ribbed Scarf,https://images4-e.ravelrycache.com/uploads/Ply...,JoAnne Turcotte,https://images4-e.ravelrycache.com/uploads/Ply...,NaN,jpg,"(279164, 'https://images4-e.ravelrycache.com/u..."
3,"{""patterns"": {""934524"":{""comments_count"":1,""cr...",934524,Garter Stitch Bonnet,NaN,Yarnspirations Design Studio,NaN,NaN,jpg,"(934524, nan, 'jpg')"
4,"{""patterns"": {""934526"":{""comments_count"":0,""cr...",934526,Baby Turban,NaN,Yarnspirations Design Studio,NaN,NaN,jpg,"(934526, nan, 'jpg')"


In [62]:
# extract category (technically Rav's child category, because Hat parent category is pointless) from patterns column
d['category'] = d['patterns'].str.extract('"pattern_categories":[{"id":[0-9]+,"name":"([^\s"]+(?:\s+[^\s"]+){,9})"', expand=False)

In [63]:
d.head(5)

,patterns,pattern_id,pattern_name,medium2_urls,author_name,medium_url,remainder,extension,ids_urls,category
0,"{""patterns"": {""17016"":{""comments_count"":0,""cre...",17016,13505 Bonnet,https://images4-e.ravelrycache.com/uploads/mam...,Dale Design,https://images4-e.ravelrycache.com/uploads/mam...,"""small2_url"":""https://images4-b.ravelrycache.c...",jpg,"(17016, 'https://images4-e.ravelrycache.com/up...",Bonnet
1,"{""patterns"": {""541305"":{""comments_count"":0,""cr...",541305,Alfa lue og hals,https://images4-b.ravelrycache.com/uploads/Gur...,Guri Østereng Halvorsen,https://images4-b.ravelrycache.com/uploads/Gur...,NaN,JPG,"(541305, 'https://images4-b.ravelrycache.com/u...",Cowl
2,"{""patterns"": {""279164"":{""comments_count"":0,""cr...",279164,F145 Heaven Ribbed Scarf,https://images4-e.ravelrycache.com/uploads/Ply...,JoAnne Turcotte,https://images4-e.ravelrycache.com/uploads/Ply...,NaN,jpg,"(279164, 'https://images4-e.ravelrycache.com/u...",Scarf
3,"{""patterns"": {""934524"":{""comments_count"":1,""cr...",934524,Garter Stitch Bonnet,NaN,Yarnspirations Design Studio,NaN,NaN,jpg,"(934524, nan, 'jpg')",Bonnet
4,"{""patterns"": {""934526"":{""comments_count"":0,""cr...",934526,Baby Turban,NaN,Yarnspirations Design Studio,NaN,NaN,jpg,"(934526, nan, 'jpg')",Other


In [64]:
# extract list of pattern attributes from patterns column
d['attribute'] = d['patterns'].str.extract('"pattern_attributes":\[(.*)\],"', expand=False)

In [65]:
d.head()

,patterns,pattern_id,pattern_name,medium2_urls,author_name,medium_url,remainder,extension,ids_urls,category,attribute
0,"{""patterns"": {""17016"":{""comments_count"":0,""cre...",17016,13505 Bonnet,https://images4-e.ravelrycache.com/uploads/mam...,Dale Design,https://images4-e.ravelrycache.com/uploads/mam...,"""small2_url"":""https://images4-b.ravelrycache.c...",jpg,"(17016, 'https://images4-e.ravelrycache.com/up...",Bonnet,"{""id"":3,""permalink"":""unisex""},{""id"":4,""permali..."
1,"{""patterns"": {""541305"":{""comments_count"":0,""cr...",541305,Alfa lue og hals,https://images4-b.ravelrycache.com/uploads/Gur...,Guri Østereng Halvorsen,https://images4-b.ravelrycache.com/uploads/Gur...,NaN,JPG,"(541305, 'https://images4-b.ravelrycache.com/u...",Cowl,"{""id"":212,""permalink"":""bottom-up""},{""id"":286,""..."
2,"{""patterns"": {""279164"":{""comments_count"":0,""cr...",279164,F145 Heaven Ribbed Scarf,https://images4-e.ravelrycache.com/uploads/Ply...,JoAnne Turcotte,https://images4-e.ravelrycache.com/uploads/Ply...,NaN,jpg,"(279164, 'https://images4-e.ravelrycache.com/u...",Scarf,"{""id"":10,""permalink"":""adult""},{""id"":64,""permal..."
3,"{""patterns"": {""934524"":{""comments_count"":1,""cr...",934524,Garter Stitch Bonnet,NaN,Yarnspirations Design Studio,NaN,NaN,jpg,"(934524, nan, 'jpg')",Bonnet,"{""id"":3,""permalink"":""unisex""},{""id"":4,""permali..."
4,"{""patterns"": {""934526"":{""comments_count"":0,""cr...",934526,Baby Turban,NaN,Yarnspirations Design Studio,NaN,NaN,jpg,"(934526, nan, 'jpg')",Other,"{""id"":3,""permalink"":""unisex""},{""id"":4,""permali..."


In [66]:
# if X in attributes column put a 1 in attribute, if not a 0
# d[att]= d["attribute"].str.find(att)
# att = "felted"

atts = ["felted", "tassel", "stripes", "bobble-or-popcorn", "cables", "eyelets", "lace", "newborn", "adult", 
        "baby", "child", "toddler", "ribbed", "slipped-stitches", "textured", "twisted-stitches", 
        "fairisle", "stranded", "appliqued", "norwegian"]

for att in atts:
    d[att] = d["attribute"].str.find(att)

In [67]:
d.head()

,patterns,pattern_id,pattern_name,medium2_urls,author_name,medium_url,remainder,extension,ids_urls,category,...,child,toddler,ribbed,slipped-stitches,textured,twisted-stitches,fairisle,stranded,appliqued,norwegian
0,"{""patterns"": {""17016"":{""comments_count"":0,""cre...",17016,13505 Bonnet,https://images4-e.ravelrycache.com/uploads/mam...,Dale Design,https://images4-e.ravelrycache.com/uploads/mam...,"""small2_url"":""https://images4-b.ravelrycache.c...",jpg,"(17016, 'https://images4-e.ravelrycache.com/up...",Bonnet,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,"{""patterns"": {""541305"":{""comments_count"":0,""cr...",541305,Alfa lue og hals,https://images4-b.ravelrycache.com/uploads/Gur...,Guri Østereng Halvorsen,https://images4-b.ravelrycache.com/uploads/Gur...,NaN,JPG,"(541305, 'https://images4-b.ravelrycache.com/u...",Cowl,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,"{""patterns"": {""279164"":{""comments_count"":0,""cr...",279164,F145 Heaven Ribbed Scarf,https://images4-e.ravelrycache.com/uploads/Ply...,JoAnne Turcotte,https://images4-e.ravelrycache.com/uploads/Ply...,NaN,jpg,"(279164, 'https://images4-e.ravelrycache.com/u...",Scarf,...,-1,-1,52,-1,-1,-1,-1,-1,-1,-1
3,"{""patterns"": {""934524"":{""comments_count"":1,""cr...",934524,Garter Stitch Bonnet,NaN,Yarnspirations Design Studio,NaN,NaN,jpg,"(934524, nan, 'jpg')",Bonnet,...,-1,79,-1,-1,-1,-1,-1,-1,-1,-1
4,"{""patterns"": {""934526"":{""comments_count"":0,""cr...",934526,Baby Turban,NaN,Yarnspirations Design Studio,NaN,NaN,jpg,"(934526, nan, 'jpg')",Other,...,-1,79,-1,-1,-1,-1,-1,-1,-1,-1


In [68]:
att_columns = ["felted", "tassel", "stripes", "bobble-or-popcorn", "cables", "eyelets", "lace", "newborn", "adult", 
    "baby", "child", "toddler", "ribbed", "slipped-stitches", "textured", "twisted-stitches", 
    "fairisle", "stranded", "appliqued", "norwegian"]

for att in atts:
    d[att] = d[att].replace(-1, 0)

In [69]:
for att in att_columns:
    d.loc[d[att] != 0, att] = 1

In [53]:
d.head()

,patterns,pattern_id,pattern_name,medium2_urls,author_name,medium_url,remainder,extension,ids_urls,category,...,child,toddler,ribbed,slipped-stitches,textured,twisted-stitches,fairisle,stranded,appliqued,norwegian
0,"{""patterns"": {""276076"":{""comments_count"":0,""cr...",2776,Hoodwink - Hooded infinity scarf,https://images4-b.ravelrycache.com/uploads/Art...,Artesanitarium Designs,https://images4-b.ravelrycache.com/uploads/Art...,"""small2_url"":""https://images4-e.ravelrycache.c...",jpg,"('2776', 'https://images4-b.ravelrycache.com/u...",Scarf,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"{""patterns"": {""538220"":{""comments_count"":0,""cr...",5382,Adelia Beanie,https://images4-b.ravelrycache.com/uploads/The...,Lauren Klipp,https://images4-b.ravelrycache.com/uploads/The...,NaN,jpg,"('5382', 'https://images4-b.ravelrycache.com/u...","Beanie, Toque",...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"{""patterns"": {""931435"":{""comments_count"":0,""cr...",931435,Fox Hat and Mittens,https://images4-b.ravelrycache.com/uploads/luc...,Lucy Chaplin,https://images4-b.ravelrycache.com/uploads/luc...,NaN,jpeg,"('931435', 'https://images4-b.ravelrycache.com...",Mittens,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,"{""patterns"": {""800367"":{""comments_count"":0,""cr...",367,gavroche hat,https://images4-b.ravelrycache.com/uploads/pal...,Palaluna,https://images4-b.ravelrycache.com/uploads/pal...,"""small2_url"":""https://images4-b.ravelrycache.c...",JPG,"('0367', 'https://images4-b.ravelrycache.com/u...","Beanie, Toque",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"{""patterns"": {""13936"":{""comments_count"":0,""cre...",13936,Tam,https://images4-b.ravelrycache.com/uploads/Woo...,Charlene Schurch,https://images4-b.ravelrycache.com/uploads/Woo...,NaN,jpg,"('13936', 'https://images4-b.ravelrycache.com/...","Beret, Tam",...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
d.drop(['patterns', 'medium2_urls', 'author_name', 'medium_url', 'remainder', 'extension', 'ids_urls'], axis=1, inplace=True)

In [71]:
d.head()

,pattern_id,pattern_name,category,attribute,felted,tassel,stripes,bobble-or-popcorn,cables,eyelets,...,child,toddler,ribbed,slipped-stitches,textured,twisted-stitches,fairisle,stranded,appliqued,norwegian
0,17016,13505 Bonnet,Bonnet,"{""id"":3,""permalink"":""unisex""},{""id"":4,""permali...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,541305,Alfa lue og hals,Cowl,"{""id"":212,""permalink"":""bottom-up""},{""id"":286,""...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,279164,F145 Heaven Ribbed Scarf,Scarf,"{""id"":10,""permalink"":""adult""},{""id"":64,""permal...",0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,934524,Garter Stitch Bonnet,Bonnet,"{""id"":3,""permalink"":""unisex""},{""id"":4,""permali...",0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,934526,Baby Turban,Other,"{""id"":3,""permalink"":""unisex""},{""id"":4,""permali...",0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [72]:
new_csv_filename = '/Users/ssivek/Insight_Projects/Ravelry_Project/Data/cat_att_csvs/cat_att_csv' + str(which_file) + '.csv'
d.to_csv(new_csv_filename, index=False)